In [12]:
from copy import deepcopy
from pathlib import Path

import pandas as pd

import lib

pd.set_option("display.max_rows", 1000)

In [13]:
GESTURE = "gesture"
CHURN = "churn"
EYE = "eye"
CALIFORNIA = "california"
HOUSE = "house"
ADULT = "adult"
OTTO = "otto"
HIGGS_SMALL = "higgs-small"
FB_COMMENTS = "fb-comments"
SANTANDER = "santander"
COVTYPE = "covtype"
MICROSOFT = "microsoft"
DATASETS = [
    GESTURE,
    CHURN,
    EYE,
    CALIFORNIA,
    HOUSE,
    ADULT,
    OTTO,
    HIGGS_SMALL,
    FB_COMMENTS,
    SANTANDER,
    COVTYPE,
    MICROSOFT,
]

DETAILS = ["task_type", "n_objects", "n_features"]


def make_datasets_df():
    df = pd.DataFrame.from_records(list(map(lib.data.load_dataset_info, DATASETS)))
    df = df.sort_values("size")
    df = df.reset_index(drop=True)
    df = df[
        [
            "name",
            "size",
            "n_features",
            "n_num_features",
            "n_cat_features",
            "task_type",
            "n_classes",
        ]
    ]
    return df


def collect_outputs(dir_, n_seeds):
    dir_ = lib.get_path(dir_)
    if dir_.exists():
        return [
            x
            for x in dir_.iterdir()
            if (
                x.is_dir()
                and (x / "DONE").exists()
                and x.name.isdigit()
                and int(x.name) < n_seeds
            )
        ]
    else:
        return []


def load_record(output, key, subkey):
    output = lib.get_path(output)
    report = lib.load_report(output)
    if Path(report["program"]).stem == "tune":
        report = report["best"]

    if Path(report["program"]).stem == "ensemble":
        program = report["single_model_program"]
        data_path = report["data"]
    else:
        program = report["program"]
        data_path = report["config"]["data"]["path"]
    dataset = Path(data_path).name

    dataset_info = lib.load_dataset_info(dataset)
    record = {
        "dataset": dataset_info["name"],
        "task_type": dataset_info["task_type"],
        "n_objects": dataset_info["size"],
        "n_features": dataset_info["n_num_features"] + dataset_info["n_cat_features"],
        "key": (
            f"{Path(program).stem} | {output.relative_to(lib.PROJ).parent.name}"
            if key is None
            else key
        ),
        "subkey": subkey,
    }
    for part in lib.Part:
        part = part.value
        if part in report["metrics"]:
            score = report["metrics"][part]["score"]
            if dataset_info["id"] == "house--default":
                score /= 10000
            record[f"{part}_score"] = score
    return record


def sort(df, by):
    if isinstance(by, str):
        by = [by]
    return df.sort_values(
        ["n_objects", "dataset"] + by,
        ascending=[True, True] + ["score" not in x for x in by],
    ).reset_index(drop=True)


def make_df(records):
    df = pd.DataFrame(records)
    if not df["subkey"].any():
        df.drop(columns=["subkey"], inplace=True)
    return sort(df, "val_score").reset_index(drop=True)


def format_scores(df, precision):
    def f(record):
        if record["task_type"] == lib.TaskType.REGRESSION.value:
            for part in lib.Part:
                for suffix in "best", "score":
                    key = f"{part.value}_{suffix}"
                    if key in record:
                        record[key] *= -1
        for k, v in list(record.items()):
            if isinstance(v, float):
                record[k] = round(v, precision)
        return record

    return df.apply(f, axis=1)


def drop_details(df):
    return df.drop(columns=DETAILS)


def drop_std(df):
    return df.drop(columns=[x for x in df.columns if x.endswith("_std")])


def build_df(records_info, precision=None, details=True):
    # (sub_)key_fn: Union[None, str, Callable[[output_dir], str]]
    records = []
    for dir_, output_filter_fn, key_fn, subkey_fn in records_info:
        for output in collect_outputs(dir_, output_filter_fn):
            key, subkey = [
                None if fn is None else fn if isinstance(fn, str) else fn(output)
                for fn in [key_fn, subkey_fn]
            ]
            record = load_record(output, key, subkey)
            if record is not None:
                records.append(record)
    if not records:
        raise RuntimeError("No records are available!")
    df = make_df(records)
    if precision is not None:
        df = format_scores(df, precision)
    if not details:
        df = drop_details(df)
    return df


def aggregate(df):
    aggrs = dict(
        test_score=("test_score", "mean"),
        test_std=("test_score", "std"),
        val_score=("val_score", "mean"),
        val_std=("val_score", "std"),
        train_score=("train_score", "mean"),
        train_std=("train_score", "std"),
        count=("test_score", "count"),
    )
    for x in DETAILS:
        if x in df.columns:
            aggrs[x] = (x, "first")
    df = df.groupby(["dataset", "key"]).agg(**aggrs)
    df["count"] = df["count"].astype(int)
    return df.reset_index().fillna(0.0)

In [17]:
# Use these flags to turn on/off the results for single models and ensembles.
# Results for ensembles are marked with '(e)'.
single_models = False
ensembles = True

results_info = []
for dataset in [
    GESTURE,
    CHURN,
    EYE,
    CALIFORNIA,
    HOUSE,
    ADULT,
    OTTO,
    HIGGS_SMALL,
    FB_COMMENTS,
    SANTANDER,
    COVTYPE,
    MICROSOFT,
]:
    for algorithm_info in [
        # (directory name, formatted name, experiment "names")
        # example: ('mlp', 'MLP', [0, 1, 'two', 'hello-world']),
        ("xgboost_", "XGBoost", [0]),
        ("catboost_", "CatBoost", [0]),
        ("mlp", "MLP", [0]),
        ("mlp-lr", "MLP-LR", [0]),
        ("mlp-q-lr", "MLP-Q-LR", [0]),
        ("mlp-t-lr", "MLP-T-LR", [0]),
        ("mlp-plr", "MLP-PLR", [0]),
        ("resnet", "ResNet", [0]),
        ("resnet-lr", "ResNet-LR", [0]),
        ("resnet-q-lr", "ResNet-Q-LR", [0]),
        ("resnet-t-lr", "ResNet-T-LR", [0]),
        ("resnet-plr", "ResNet-PLR", [0]),
        ("transformer-l", "Transformer-l", [0]),
        ("transformer-lr", "Transformer-LR", [0]),
        ("transformer-q-lr", "Transformer-Q-LR", [0]),
        ("transformer-t-lr", "Transformer-T-LR", [0]),
        ("transformer-plr", "Transformer-PLR", [0]),
    ]:
        if isinstance(algorithm_info, str):
            algorithm_info = (algorithm_info, algorithm_info, [0])
        alg_dir, name, experiments = algorithm_info
        if name is None:
            name = alg_dir

        for experiment in experiments:
            prefix = name if experiment == 0 else f"[{experiment}] {name}"
            if single_models:
                results_info.append(
                    (
                        lib.EXP / alg_dir / dataset / f"{experiment}_evaluation",
                        15,
                        prefix,
                        lambda x: x.name,
                    )
                )
            if ensembles:
                results_info.append(
                    (
                        lib.EXP / alg_dir / dataset / f"{experiment}_ensemble_5",
                        3,
                        prefix + " (e)",
                        lambda x: x.name,
                    )
                )

df = build_df(results_info)
df = aggregate(df)
df = sort(df, "test_score")
df = format_scores(df, 4)
df = df.set_index(["dataset", "key"])

df_ranks = deepcopy(df).reset_index()
df_ranks.loc[df_ranks["task_type"] == "regression", "test_score"] *= -1
df_ranks = df_ranks.pivot("key", "dataset", "test_score")
df_ranks.columns = df_ranks.columns.map(lambda x: x.split()[0].split("-")[0])
df_ranks = df_ranks.rank(0, ascending=False)
df_ranks["avg"] = df_ranks.mean(1)
df_ranks["std"] = df_ranks.std(1)
df_ranks.insert(0, "avg", df_ranks.pop("avg"))
df_ranks.insert(1, "std", df_ranks.pop("std"))
df_ranks = df_ranks.sort_values("avg")

# df = drop_details(df)

display(make_datasets_df())
display(df_ranks)
display(df)

,name,size,n_features,n_num_features,n_cat_features,task_type,n_classes
0,Gesture Phase,9873,32,32,0,multiclass,5.0
1,Churn Modelling,10000,11,10,1,binclass,NaN
2,Eye Movements,10936,26,26,0,multiclass,3.0
3,California Housing,20640,8,8,0,regression,NaN
4,House 16H,22784,16,16,0,regression,NaN
5,Adult,48842,14,6,8,binclass,NaN
6,Otto Group Products,61878,93,93,0,multiclass,9.0
7,Higgs Small,98049,28,28,0,binclass,NaN
8,Facebook Comments Volume,197080,51,50,1,regression,NaN
9,Santander Customer Transactions,200000,200,200,0,binclass,NaN


dataset,avg,std,Adult,California,Churn,Covertype,Eye,Facebook,Gesture,Higgs,House,MSLR,Otto,Santander
key,,,,,,,,,,,,,,
MLP-PLR (e),4.916667,4.609019,3.0,13.0,15.0,6.0,2.0,5.0,1.0,1.0,1.0,8.0,1.0,3.0
ResNet-PLR (e),5.208333,2.221658,2.0,9.0,9.5,4.5,3.0,6.0,3.0,4.0,5.0,5.0,4.5,7.0
Transformer-PLR (e),5.958333,4.351524,4.0,11.0,1.0,7.0,1.0,16.0,6.5,3.0,9.0,3.0,8.0,2.0
CatBoost (e),6.791667,4.656798,5.0,5.0,7.5,17.0,10.0,2.0,2.0,13.0,10.0,1.0,4.5,4.5
Transformer-T-LR (e),7.166667,4.403282,7.0,1.0,2.0,1.0,5.0,15.0,6.5,6.5,14.0,10.5,9.0,8.5
ResNet-T-LR (e),7.625000,3.464853,6.0,2.5,4.0,4.5,6.0,11.5,9.0,8.0,4.0,12.0,11.0,13.0
Transformer-Q-LR (e),8.583333,5.271122,9.0,2.5,16.0,3.0,4.0,11.5,4.0,15.0,13.0,9.0,15.0,1.0
XGBoost (e),9.000000,5.461990,1.0,7.0,13.0,14.0,11.0,4.0,8.0,16.0,15.0,2.0,2.0,15.0
ResNet-Q-LR (e),9.166667,4.579544,10.0,4.0,12.0,2.0,7.0,3.0,12.0,10.0,6.0,15.0,17.0,12.0


test_score  test_std  \
dataset                         key                                          
Gesture Phase                   MLP-PLR (e)               0.6999    0.0026   
                                CatBoost (e)              0.6920    0.0023   
                                ResNet-PLR (e)            0.6910    0.0078   
                                Transformer-Q-LR (e)      0.6903    0.0023   
                                ResNet (e)                0.6900    0.0072   
                                Transformer-PLR (e)       0.6864    0.0076   
                                Transformer-T-LR (e)      0.6864    0.0051   
                                XGBoost (e)               0.6829    0.0016   
                                ResNet-T-LR (e)           0.6825    0.0075   
                                MLP-Q-LR (e)              0.6820    0.0048   
                                MLP-LR (e)                0.6793    0.0060   
                                ResNet-Q-LR (e)           0.6744    0.0031   
                                MLP-T-LR (e)              0.6734    0.0010   
                                ResNet-LR (e)             0.6724    0.0073   
                                Transformer-l (e)         0.6678    0.0157   
                                Transformer-LR (e)        0.6657    0.0013   
                                MLP (e)                   0.6651    0.0033   
Churn Modelling                 Transformer-PLR (e)       0.8643    0.0012   
                                Transformer-T-LR (e)      0.8625    0.0013   
                                ResNet-LR (e)             0.8617    0.0021   
                                ResNet-T-LR (e)           0.8615    0.0000   
                                MLP-LR (e)                0.8613    0.0012   
                                MLP-T-LR (e)              0.8613    0.0010   
                                CatBoost (e)              0.8612    0.0003   
                                Transformer-l (e)         0.8612    0.0008   
                                ResNet-PLR (e)            0.8610    0.0005   
                                Transformer-LR (e)        0.8610    0.0005   
                                ResNet (e)                0.8608    0.0020   
                                ResNet-Q-LR (e)           0.8590    0.0022   
                                XGBoost (e)               0.8588    0.0003   
                                MLP-Q-LR (e)              0.8587    0.0006   
                                MLP-PLR (e)               0.8575    0.0020   
                                Transformer-Q-LR (e)      0.8572    0.0003   
                                MLP (e)                   0.8563    0.0014   
Eye Movements                   Transformer-PLR (e)       0.9767    0.0048   
                                MLP-PLR (e)               0.9676    0.0072   
                                ResNet-PLR (e)            0.9247    0.0094   
                                Transformer-Q-LR (e)      0.8423    0.0075   
                                Transformer-T-LR (e)      0.8335    0.0078   
                                ResNet-T-LR (e)           0.8173    0.0025   
                                ResNet-Q-LR (e)           0.7937    0.0146   
                                Transformer-LR (e)        0.7756    0.0056   
                                Transformer-l (e)         0.7693    0.0077   
                                CatBoost (e)              0.7570    0.0028   
                                XGBoost (e)               0.7381    0.0069   
                                ResNet-LR (e)             0.7351    0.0037   
                                MLP-Q-LR (e)              0.7316    0.0048   
                                MLP-T-LR (e)              0.7290    0.0059   
                                MLP-LR (e)                0.6938    0.0079   
                                ResNet (e)                0.6667    0.0064  